# PipekRetrainedH

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.PipekRetrainedH)

class PipekRetrainedH(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        """
        Applies an anti-logarithmic linear transformation to a PyTorch tensor.
        """
        adult_age = 20

        # Create a mask for negative and non-negative values
        mask_negative = x < 0
        mask_non_negative = ~mask_negative

        # Initialize the result tensor
        age_tensor = torch.empty_like(x)

        # Exponential transformation for negative values
        age_tensor[mask_negative] = (1 + adult_age) * torch.exp(x[mask_negative]) - 1

        # Linear transformation for non-negative values
        age_tensor[mask_non_negative] = (1 + adult_age) * x[mask_non_negative] + adult_age

        return age_tensor



In [3]:
model = pya.models.PipekRetrainedH()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'pipekretrainedh'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2022
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = "Pipek, Orsolya Anna, and István Csabai. \"A revised multi-tissue, multi-platform epigenetic clock model for methylation array data.\" Journal of Mathematical Chemistry 61.2 (2023): 376-388."
model.metadata["doi"] = "https://doi.org/10.1007/s10910-022-01381-4"
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download directly with curl

In [5]:
github_url = "https://github.com/pipekorsi/MepiClock.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

## Load features

#### From CSV file

In [7]:
df = pd.read_csv('MepiClock/resources/model_coefficients.csv', sep=';')
df['feature'] = df['probeID']
df['coefficient'] = df['retrained H (308)']
df = df[df['coefficient'] != 0]
model.features = df['feature'][1:].tolist()

## Load weights into base model

In [8]:
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

#### Linear model

In [9]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

#### From CSV file

In [10]:
model.reference_values = None

## Load preprocess and postprocess objects

In [11]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [12]:
model.postprocess_name = 'anti_log_linear'
model.postprocess_dependencies = None

## Check all clock parameters

In [13]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'Pipek, Orsolya Anna, and István Csabai. "A revised multi-tissue, '
             'multi-platform epigenetic clock model for methylation array '
             'data." Journal of Mathematical Chemistry 61.2 (2023): 376-388.',
 'clock_name': 'pipekretrainedh',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1007/s10910-022-01381-4',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2022}
reference_values: None
preprocess_name: None
preprocess_dependencies: None
postprocess_name: 'anti_log_linear'
postprocess_dependencies: None
features: ['cg00075967', 'cg00091693', 'cg00168942', 'cg00374717', 'cg00431549', 'cg00436603', 'cg00864867', 'cg00945507', 'cg01027739', 'cg01027805', 'cg01262913', 'cg01353448', 'cg01407797', 'cg01459453', 'cg01485645', 'cg01511567', 'cg015

## Basic test

In [14]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ 36.4307],
        [ -0.9986],
        [ -0.8102],
        [ -0.9982],
        [ 63.8602],
        [ -1.0000],
        [ -1.0000],
        [237.1113],
        [247.2407],
        [229.1479]], dtype=torch.float64, grad_fn=<IndexPutBackward0>)

## Save torch model

In [15]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [16]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: reference_feature_values.csv
Deleted folder: MepiClock
